Using GPT2 to encode the text

In [1]:
!pip install transformers


[notice] A new release of pip available: 22.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer,GPT2Model
import scipy.io as sio
import numpy as np
import torch


Load GPT-2 model

In [3]:
# model = GPT2LMHeadModel.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2-medium')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium') #tokeniser
# word_embeddings = model.transformer.wte.weight  # Word Token Embeddings
# position_embeddings = model.transformer.wpe.weight  # Word Position Embeddings

c:\users\portable\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Portable\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Convert dataset to tokens

In [4]:
!pip install patool

     ---------------------------------------- 91.1/91.1 kB 1.7 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import patoolib

In [9]:
patoolib.extract_archive("doi_10.5061_dryad.gt413__v1.zip")

PatoolError: file `/doi_10.5061_dryad.gt413__v1.zip' was not found

In [10]:
data = sio.loadmat('./doi_10.5061_dryad.gt413__v1/subject_1.mat') #load subject 1 data

In [11]:
# extract words from array
vocab = []
content = data.get('words')
for word in content[0]:
  vocab.append(word[0][0][0][0])

In [12]:
vocab

['Harry',
 'had',
 'never',
 'believed',
 'he',
 'would',
 'meet',
 'a',
 'boy',
 'he',
 'hated',
 'more',
 'than',
 'Dudley,',
 'but',
 'that',
 'was',
 'before',
 'he',
 'met',
 'Draco',
 'Malfoy.',
 'Still,',
 'first-year',
 'Gryffindors',
 'only',
 'had',
 'Potions',
 'with',
 'the',
 'Slytherins,',
 'so',
 'they',
 "didn't",
 'have',
 'to',
 'put',
 'up',
 'with',
 'Malfoy',
 'much.',
 'Or',
 'at',
 'least,',
 'they',
 "didn't",
 'until',
 'they',
 'spotted',
 'a',
 'notice',
 'pinned',
 'up',
 'in',
 'the',
 'Gryffindor',
 'common',
 'room',
 'that',
 'made',
 'them',
 'all',
 'groan.',
 'Flying',
 'lessons',
 'would',
 'be',
 'starting',
 'on',
 'Thursday',
 '--',
 'and',
 'Gryffindor',
 'and',
 'Slytherin',
 'would',
 'be',
 'learning',
 'together.',
 '+',
 '"Typical,"',
 'said',
 'Harry',
 'darkly.',
 '"Just',
 'what',
 'I',
 'always',
 'wanted.',
 'To',
 'make',
 'a',
 'fool',
 'of',
 'myself',
 'on',
 'a',
 'broomstick',
 'in',
 'front',
 'of',
 'Malfoy."',
 '+',
 'He',
 'ha

In [13]:
# Initialize a list to store the separated strings
processed_vocab = []

# Iterate through the vocabulary and split on encountering '+'
current_string = []
for word in vocab:
    if word == '+':
        if current_string:
            processed_vocab.append(' '.join(current_string))
            current_string = []
    else:
        current_string.append(word)

# Add the last separated string if any words are left
if current_string:
    processed_vocab.append(', '.join(current_string))


In [14]:
# Print the result
processed_vocab

["Harry had never believed he would meet a boy he hated more than Dudley, but that was before he met Draco Malfoy. Still, first-year Gryffindors only had Potions with the Slytherins, so they didn't have to put up with Malfoy much. Or at least, they didn't until they spotted a notice pinned up in the Gryffindor common room that made them all groan. Flying lessons would be starting on Thursday -- and Gryffindor and Slytherin would be learning together.",
 '"Typical," said Harry darkly. "Just what I always wanted. To make a fool of myself on a broomstick in front of Malfoy."',
 'He had been looking forward to learning to fly more than anything else. "You don\'t know that you\'ll make a fool of yourself," said Ron reasonably. "Anyway, I know Malfoy\'s always going on about how good he is at Quidditch, but I bet that\'s all talk."',
 "Malfoy certainly did talk about flying a lot. He complained loudly about first years never getting on the House Quidditch teams and told long, boastful storie

Tokenize vocab using GPT-2 tokenizer

In [8]:
# Set a padding token
tokenizer.pad_token = tokenizer.eos_token

# tokenize vocab
vocab_tokens = tokenizer(processed_vocab, return_tensors='pt', padding=True, truncation=True)

In [9]:
len(vocab_tokens)

2

In [ ]:
with torch.no_grad():
    outputs = model(**vocab_tokens)

In [ ]:
contextual_embeddings = outputs.last_hidden_state
# Print the shape of the contextual embeddings
print(contextual_embeddings.shape)

In [15]:
contextual_embeddings

tensor([[[-2.8540e-01, -1.6913e-01,  7.1526e-02,  ..., -4.4242e-01,
          -4.2040e-01, -4.5144e-01],
         [ 1.9276e-01,  5.3168e-02,  8.0529e-02,  ..., -2.3211e-01,
          -3.2854e-01, -1.0171e-01],
         [ 1.7249e-01,  4.2546e-03,  1.0617e-01,  ..., -2.1924e-01,
          -3.5968e-01, -1.1085e-01],
         ...,
         [ 1.7135e-01, -5.0547e-03,  1.1724e-01,  ..., -2.1587e-01,
          -3.6111e-01, -1.1104e-01],
         [ 1.7183e-01, -8.1839e-03,  1.2183e-01,  ..., -2.1472e-01,
          -3.6065e-01, -1.1169e-01],
         [ 1.7230e-01, -1.0424e-02,  1.2564e-01,  ..., -2.1379e-01,
          -3.6049e-01, -1.1189e-01]],

        [[ 6.6167e-02, -4.4810e-01, -3.0093e-01,  ..., -2.5207e-01,
           2.1428e-01, -1.2771e-01],
         [ 1.7104e-01,  1.5008e-02,  2.7429e-02,  ..., -1.7947e-01,
          -3.1431e-01, -8.3762e-02],
         [ 1.5827e-01, -7.3258e-03,  4.5117e-02,  ..., -1.8052e-01,
          -3.3583e-01, -8.5080e-02],
         ...,
         [ 1.5753e-01, -1

In [36]:
word_to_find = "Draco"
word_index = vocab.index(word_to_find)
word_index

2

In [37]:
# Extract the embedding for a word
word_embedding = contextual_embeddings[0, word_index]

In [38]:
len(word_embedding)

1024